# About this Project
As an Applied Data Science student focusing in Astronomy, the excitment of being able to combine the both fields to find Earth like planets (maybe the pontential for life) from data gather by telescopes is inspiring and motivating.

This is my attempt at analyzing this dataset.

Reference: https://www.kaggle.com/eduardowoj/exoplanets-database?select=kepler.csv

# Context
Extrasolar planets, or exoplanets for short, are all the fuss in astronomy these days. With classes in full swing and professor teaching us about all the possiblities out their, analysing a dataset like this is really exiting and relevant.

The data is well structured in a CSV file, but since it comes from several different sources, some parameters aren't well formated, and require some cleaning and filtering, an additional challenge!

# Acknowledgements
http://exoplanet.eu/


In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px;
from matplotlib import rc
import warnings
warnings.filterwarnings('default')

In [ ]:
data = pd.read_csv("../input/exoplanets-database/kepler.csv")
data.head(3)
print(data.size)

In [ ]:
# dropping unnecessary columns for EDA
dropped_data = data.drop(columns=["mass_sini","mass_error_min", "mass_error_max", "mass_sini_error_min", "mass_sini_error_max", "radius_error_min",
                          "radius_error_max", "orbital_period_error_min", "orbital_period_error_max", "semi_major_axis_error_min",
                          "semi_major_axis_error_max", "eccentricity_error_min", "eccentricity_error_max", "inclination_error_min",
                          "inclination_error_max", "omega_error_min", "omega_error_max", "tperi_error_min", "tperi_error_max",
                          "tconj_error_min", "tconj_error_max", "tzero_tr_error_min", "tzero_tr_error_max", "tzero_tr_sec_error_min",
                          "geometric_albedo_error_min", "geometric_albedo_error_max", "mass_detection_type", "star_distance_error_min",
                          "star_distance_error_max", "star_metallicity_error_min", "star_metallicity_error_max", "star_mass_error_min",
                          "star_mass_error_max", "star_radius_error_min", "star_radius_error_max", "star_age_error_min", "star_age_error_max",
                          "star_teff_error_min", "star_teff_error_max", "tzero_tr_sec_error_max", "lambda_angle_error_min", "lambda_angle_error_max",
                          "impact_parameter_error_min", "impact_parameter_error_max", "tzero_vr_error_min", "tzero_vr_error_max", "k_error_min",
                          "k_error_max", "temp_calculated_error_min", "temp_calculated_error_max", "semi_major_axis", "inclination",
                          "omega", "tperi", "tconj", "tzero_tr", "tzero_tr_sec", "lambda_angle", "impact_parameter", "tzero_vr", "k", "hot_point_lon",
                          "geometric_albedo",	"log_g", "alternate_names", "ra",	"dec", "mag_v",	"mag_i", "mag_j",	"mag_h", "mag_k"])
dropped_data.head(3)

In [ ]:
dropped_data["planet_status"].unique

In [ ]:
dropped_data.size

In [ ]:
dropped_data.isnull().sum()

In [ ]:
# Molecules found
dropped_data.molecules.unique()

## EDA
### PLANETS

#### **Looking for any relationship between significant variables**

In [ ]:
# Relationship between Mass and Orbital Period
fig, ax = plt.subplots(figsize = (10,10))
scatter = ax.scatter(
    x = dropped_data["mass"],
    y = dropped_data["orbital_period"]
)
plt.xlabel("Mass of Exoplanets")
plt.ylabel("Orbital Period")
plt.title("Mass of Planet and it's Orbital Period")
;

No relationship between between Mass and Orbital Period of the planets

In [ ]:
# Relationship between Mass and Radius of Planets
sns.lmplot(x = "mass", y = "radius", data = dropped_data);

No relationship between Mass and Radius of the planets

In [ ]:
# when were most of the planets discorvered
plt.style.use("seaborn")

discovered = dropped_data["discovered"].value_counts()


plt.figure(figsize=(10,7))
plt.bar(discovered.index, discovered.values)
plt.xlabel("Year")
plt.ylabel("Number of Exoplanets Discovered")
plt.title("Exoplanets discovered By Year");

In [ ]:
# Which detection type found the most exoplanets

detection = dropped_data["detection_type"].value_counts()
detection.head()

fig, ax = plt.subplots(figsize = (10,7))
detection.plot.bar(color = "royalblue", alpha = 0.7)
plt.title("Detection Type and Number of Planets Found")
plt.ylabel("Number of Planets")
plt.xlabel("Detection Type")
;

In [ ]:
# Molecules found
molecules = dropped_data["molecules"].value_counts()
fig, ax = plt.subplots(figsize = (10,7))
molecules.plot.bar(color = "royalblue", alpha = 1);

### **Parameters**
1. Planet Mass
2. Planet Radius
3. Planet Molecules
4. Star Age

**To maintain a suitable atmosphere we will assume in this project that a planet should have a mass between 0.5 and 10 Earth masses, with a radius between 0.8 and 2.2 times that of Earth.**

**Furthermore, it is important that the parent star has a life span for more than 4.5 billion years and not more than 10 billion years. If the star is doesnt live for more than 4.5 billion years, life as we know may not have enough time to develop.** 

**Form what we know about life, water is essential. So, we will later look at planets within the given parameters for signs of water**

In [ ]:
# New DataFrame with mass and radius parameters
suitable = dropped_data[["mass", "radius", "star_age", "detection_type"]]

suitable = suitable[(suitable.mass >= 0.5) & (suitable.mass <= 10.0)]
suitable = suitable[(suitable.radius >= 0.8) & (suitable.radius <= 2.2)]
print(suitable.size)

suitable = suitable.dropna()
print(suitable.head(3))
print(suitable.size)

In [ ]:

plt.style.use("seaborn-dark")
fig, ax = plt.subplots(figsize = (15,10))
scatter = ax.scatter(
    x = suitable.mass,
    y = suitable.radius,
    c = suitable.star_age,
    cmap = "gist_rainbow"
);
plt.xlabel("Planet Mass")
plt.ylabel("Planet Radius")
plt.title("Mass and Radius of the Planet");

legend1 = ax.legend(*scatter.legend_elements(num=7),
                    loc="upper right", title="Star Age")
ax.add_artist(legend1);

**Since we do not know how long the parent star has existed for the 699 planets above, we will look at the planets whose calculated star age falls between 4.5 and 10 billion years**


In [ ]:
print(f"The max and min star age is: {max(suitable.star_age), min(suitable.star_age)}")
suitable = suitable[(suitable.star_age >= 4.5) & (suitable.star_age <= 10.0)]
print(f"Number of planets: {suitable.size}")
suitable.head()

In [ ]:
plt.style.use("seaborn")
fig, ax = plt.subplots(figsize = (15,10))
scatter = ax.scatter(
    x = suitable.mass,
    y = suitable.radius,
    c = suitable.star_age,
    s = 200,
    cmap = "gist_rainbow"
);
plt.xlabel("Planet Mass")
plt.ylabel("Planet Radius")
plt.title("Mass and Radius of the Planet");

legend1 = ax.legend(*scatter.legend_elements(num=5),
                    loc="upper right", title="Star Age")
ax.add_artist(legend1);

Most of the planets that have a star between the ages of 4.5 and 10 billion years are less than 4 times the mass of Earth and less than 1.4 times the radius of Earth 

In [ ]:
# scatter plot of mass and radius of planets by detection type

detection_type_numeric = {"Imaging": 1, "Radial Velocity": 2, "Primary Transit": 3, "Microlensing": 4, "Pulsar": 5, "Other": 6, "TTV": 7, "Primary Transit, TTV": 8, "Astrometry": 9}
suitable.detection_type = [detection_type_numeric[item] for item in suitable.detection_type] 


sns.set(rc={'figure.figsize':(10,5)})

sns.scatterplot(data = suitable, x = "mass", y = "radius", hue = "detection_type", style = "detection_type", s = 300);

Most of the planets within the given parameters have been found by Primary Transit

**With the given parameters for mass, radius and star age, we will find planets whose molecules have been identified.**

In [ ]:
suitable_with_mols = dropped_data[["# name", "mass", "radius", "star_age", "molecules"]]
suitable_with_mols.head()
suitable_with_mols = suitable_with_mols[(suitable_with_mols.mass >= 0.5) & (suitable_with_mols.mass <= 10.0)]
suitable_with_mols = suitable_with_mols[(suitable_with_mols.radius >= 0.8) & (suitable_with_mols.radius <= 2.2)]
suitable_with_mols = suitable_with_mols[(suitable_with_mols.star_age >= 4.5) & (suitable_with_mols.star_age <= 10.0)]
suitable_with_mols.size


In [ ]:
suitable_with_mols.molecules = suitable_with_mols.molecules.fillna("Not Given")
suitable_with_mols.head()

In [ ]:
suitable_molecules = suitable_with_mols.molecules.value_counts()
suitable_molecules.plot.bar(color = "royalblue", alpha = 1);

In [ ]:
suitables = suitable_with_mols[suitable_with_mols.molecules != "Not Given"]
suitables

**Kepler was only able to find two planets and their molecules that are in the range of the defined parameters.**

HD 80606 b has Potassium\
XO-1 b has Oxygen, Iodine, Carbon

-----------

In [ ]:
kepler_data = dropped_data[["# name", "mass", "radius",  "orbital_period", "star_name", "star_age", "molecules"]]
kepler_data = kepler_data[kepler_data['# name'].str.contains('kepler', regex=False, case=False, na=False)]
print(kepler_data.size)
kepler_data.head()


In [ ]:
max(kepler_data.orbital_period)

### **Relevant Exoplanets that were found by Kepler but not in this dataset**
1. Kepler 22b around Kepler 22
2. Kepler 186f around Kepler 186
3. Kepler 438b around Kepler 438